In [29]:
import yfinance as yahoo
import pandas as pd
import numpy as np
import scipy.optimize as sco
import copy
from scipy import stats
import datetime as dt

In [30]:
# import investment to track performance during 2020
portfolio = pd.read_excel('./excel/Elon Musk 2021-02-21.xlsx')

portfolio

,Unnamed: 0,capital,price,weights,cash,nominal,invested,percentage,total,liquid
0,ADA-USD,10000000,1.101678,0.145423,1.454226e+06,1320009,1.454225e+06,0.145437,9.999013e+06,986.829521
1,DCR-USD,10000000,167.280029,0.078917,7.891726e+05,4717,7.890599e+05,0.078914,9.999013e+06,986.829521
2,DGB-USD,10000000,0.079312,0.145423,1.454226e+06,18335426,1.454226e+06,0.145437,9.999013e+06,986.829521
3,DOGE-USD,10000000,0.054937,0.145423,1.454226e+06,26470824,1.454226e+06,0.145437,9.999013e+06,986.829521
4,ETH-USD,10000000,1954.838013,0.028624,2.862396e+05,146,2.854063e+05,0.028543,9.999013e+06,986.829521
5,LINK-USD,10000000,34.726143,0.082669,8.266851e+05,23805,8.266558e+05,0.082674,9.999013e+06,986.829521
6,MIOTA-USD,10000000,1.397013,0.013982,1.398186e+05,100083,1.398172e+05,0.013983,9.999013e+06,986.829521
7,NANO-USD,10000000,6.852354,0.091231,9.123097e+05,133138,9.123086e+05,0.091240,9.999013e+06,986.829521
8,OMG-USD,10000000,6.381282,0.062545,6.254548e+05,98013,6.254486e+05,0.062551,9.999013e+06,986.829521
9,XEM-USD,10000000,0.539113,0.112217,1.122172e+06,2081515,1.122172e+06,0.112228,9.999013e+06,986.829521


In [31]:
# Set Initial quantity to simulate first Buy-&-Hold Strategy, with nominal fixed
# Capital requested to allocate investment
initial_capital = float(input("How much will you invest? "))

How much will you invest? 10000000


In [32]:
def AdjustRisk(portfolio, comienzo, final):
  """Provide the stock list of your portfolio
     to update risk by Component-Value-at-Risk"""  
  listado = list(portfolio.index.values)
  weights = list(portfolio.weights.values)
  df = yahoo.download(listado, start=comienzo, end=final,interval="60m")['Adj Close'].fillna(method='ffill')
  data = df 
  returns = data.pct_change()
  correlation = returns.corr() # correlation
  covariance = returns.cov()  # covariance
  instruments = pd.DataFrame(index= data.columns)
  instruments['weigths'] = 1/len(instruments.index) # secure allocation is equal 1
  instruments['deltas'] = (instruments.weigths * correlation).sum() # deltas as elasticity of the assets
  instruments['Stdev'] = returns.std()
  instruments['stress'] = (instruments.deltas * instruments.Stdev) * 3 # stress applied at 4 deviations
  instruments['portfolio_stress'] = instruments.stress.sum() # the stress of the portfolio
  risk = pd.DataFrame(index=data.columns)
  risk['numerator'] = (instruments.deltas.multiply(covariance)).sum()
  risk['denominator'] = data.pct_change().std() * (-2.365)
  risk['GradVaR'] = -risk.numerator / risk.denominator
  risk['CVaRj'] = risk.GradVaR * instruments.deltas # Component VaR of the Risk Factors j
  risk['thetai'] = (risk.CVaRj * correlation).sum() # Theta i of the instruments
  risk['CVaRi'] = risk.thetai * (1/len(data.columns)) # Component VaR of the Instruments i
  risk['totalCVaRi'] = risk.CVaRi.sum() #total CVaR of the portfolio
  risk['CVaRattribution'] = risk.CVaRi / risk.totalCVaRi # risk allocation by instrument in the portfolio
  riskadj = pd.DataFrame(index=data.columns)
  riskadj['base'] = instruments['weigths'].values
  riskadj['CVaRattribution'] = risk.CVaRattribution.sort_values(axis=0,ascending=False)
  riskadj['new'] = portfolio['weights'].values  # Choosing the option with the highest return
  riskadj['condition'] = (riskadj.base / riskadj.CVaRattribution)
  riskadj['newrisk'] = (riskadj.new / riskadj.CVaRattribution)
  riskadj['differences'] = (riskadj.newrisk - riskadj.condition)  # apply this result as a percentage to multiply new weights
  riskadj['adjustments'] = (riskadj.newrisk - riskadj.condition) / riskadj.condition #ALARM if its negative sum up the difference, 
                                              #if it is positive rest it, you need to have 0
  riskadj['suggested'] = riskadj.new * (1 + riskadj.adjustments)   
  riskadj['tototal'] = riskadj.suggested.sum()
  riskadj['MinCVaR'] = riskadj.suggested / riskadj.tototal
  result = pd.DataFrame(riskadj['MinCVaR'].values,columns=['MinCVaR'],index=data.columns)
  result[result.MinCVaR>=0.12] = 0.12
  result['MinCVaR'] = result['MinCVaR'] / sum(result['MinCVaR'])
  result['lastPrice'] = (data.tail(1).T.values)
  return result

In [33]:
stocks = list(portfolio['Unnamed: 0'].values)

start, end = dt.date(2020,1,25), dt.date(2020,12,31)
df =yahoo.download(stocks,start,end)["Adj Close"].fillna(method="ffill")
best = pd.DataFrame(index=portfolio.iloc[:,0].values)
best = pd.DataFrame(index=df.columns)
best['capital'] = initial_capital
best['price'] = df.tail(1).T.values
best['weights'] = portfolio['weights'].values 
best['cash'] = (best['capital'] * best['weights'])
best['nominal'] =  best['cash'] // best['price'] 
best['invested'] = best['price'] * best['nominal']
best['percentage'] = best['invested'] / sum(best['invested'])
best['total'] = sum(best['invested'])
best['liquid'] = best['capital'] - best['total']
best = best[best.nominal!=0].dropna() # remove all stocks that you do not invest in
best


[*********************100%***********************]  12 of 12 completed


,capital,price,weights,cash,nominal,invested,percentage,total,liquid
ADA-USD,10000000.0,0.181397,0.145423,1.454226e+06,8016811.0,1.454226e+06,0.145433,9.999259e+06,741.19274
DCR-USD,10000000.0,40.776363,0.078917,7.891726e+05,19353.0,7.891450e+05,0.078920,9.999259e+06,741.19274
DGB-USD,10000000.0,0.024973,0.145423,1.454226e+06,58231918.0,1.454226e+06,0.145433,9.999259e+06,741.19274
DOGE-USD,10000000.0,0.004682,0.145423,1.454226e+06,310599261.0,1.454226e+06,0.145433,9.999259e+06,741.19274
ETH-USD,10000000.0,737.803406,0.028624,2.862396e+05,387.0,2.855299e+05,0.028555,9.999259e+06,741.19274
LINK-USD,10000000.0,11.270538,0.082669,8.266851e+05,73349.0,8.266827e+05,0.082674,9.999259e+06,741.19274
MIOTA-USD,10000000.0,0.296893,0.013982,1.398186e+05,470939.0,1.398185e+05,0.013983,9.999259e+06,741.19274
NANO-USD,10000000.0,1.021414,0.091231,9.123097e+05,893183.0,9.123097e+05,0.091238,9.999259e+06,741.19274
OMG-USD,10000000.0,2.449293,0.062545,6.254548e+05,255361.0,6.254539e+05,0.062550,9.999259e+06,741.19274
XEM-USD,10000000.0,0.204523,0.112217,1.122172e+06,5486778.0,1.122172e+06,0.112226,9.999259e+06,741.19274


# Rebalance Monthly

In [34]:
febrero,marzo = dt.date(2020,2,1), dt.date(2020,3,1)
stocks = list(best.index)
data = yahoo.download(stocks,febrero,marzo)['Adj Close'].fillna(method="ffill")

[*********************100%***********************]  12 of 12 completed


In [35]:
cartera = pd.DataFrame(best)
portfolio = pd.DataFrame(index=cartera.index)
portfolio['nominal'] = cartera['nominal'].values
portfolio['pricePaid'] = cartera['price'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = cartera['liquid'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = (yahoo.download(stocks, febrero,marzo)['Adj Close'].fillna(method='ffill')).tail(1).T
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['weights'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
febrero = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest
ADA-USD,8016811.0,0.181397,0.145433,9.999259e+06,741.19274,0.045932,3.820401e+06,0.382068,0.253213,12098786.0,4081975.0,0.145436,3.821070e+06,72.107888
DCR-USD,19353.0,40.776363,0.078920,9.999259e+06,741.19274,17.131205,3.820401e+06,0.382068,0.420126,17603.0,-1750.0,0.078920,3.821070e+06,72.107888
DGB-USD,58231918.0,0.024973,0.145433,9.999259e+06,741.19274,0.005506,3.820401e+06,0.382068,0.220478,100930165.0,42698247.0,0.145436,3.821070e+06,72.107888
DOGE-USD,310599261.0,0.004682,0.145433,9.999259e+06,741.19274,0.002275,3.820401e+06,0.382068,0.485904,244273175.0,-66326086.0,0.145436,3.821070e+06,72.107888
ETH-USD,387.0,737.803406,0.028555,9.999259e+06,741.19274,218.970596,3.820401e+06,0.382068,0.296787,498.0,111.0,0.028538,3.821070e+06,72.107888
LINK-USD,73349.0,11.270538,0.082674,9.999259e+06,741.19274,3.879533,3.820401e+06,0.382068,0.344219,81430.0,8081.0,0.082676,3.821070e+06,72.107888
MIOTA-USD,470939.0,0.296893,0.013983,9.999259e+06,741.19274,0.211438,3.820401e+06,0.382068,0.712169,252701.0,-218238.0,0.013983,3.821070e+06,72.107888
NANO-USD,893183.0,1.021414,0.091238,9.999259e+06,741.19274,0.727661,3.820401e+06,0.382068,0.712406,479113.0,-414070.0,0.091239,3.821070e+06,72.107888
OMG-USD,255361.0,2.449293,0.062550,9.999259e+06,741.19274,0.846505,3.820401e+06,0.382068,0.345612,282352.0,26991.0,0.062551,3.821070e+06,72.107888
XEM-USD,5486778.0,0.204523,0.112226,9.999259e+06,741.19274,0.048902,3.820401e+06,0.382068,0.239103,8769166.0,3282388.0,0.112228,3.821070e+06,72.107888


In [36]:
comienzo = marzo - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, marzo)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
marzo = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,12098786.0,0.045932,0.145436,3.821070e+06,72.107888,0.048052,3.940493e+06,1.031254,1.046164,12839697.0,740911.0,0.156574,3.940492e+06,73.413136,0.156571
DCR-USD,17603.0,17.131205,0.078920,3.821070e+06,72.107888,17.618406,3.940493e+06,1.031254,1.028439,17003.0,-600.0,0.076022,3.940492e+06,73.413136,0.076021
DGB-USD,100930165.0,0.005506,0.145436,3.821070e+06,72.107888,0.005769,3.940493e+06,1.031254,1.047826,106941092.0,6010927.0,0.156574,3.940492e+06,73.413136,0.156571
DOGE-USD,244273175.0,0.002275,0.145436,3.821070e+06,72.107888,0.002275,3.940493e+06,1.031254,1.000036,271189565.0,26916390.0,0.156574,3.940492e+06,73.413136,0.156571
ETH-USD,498.0,218.970596,0.028538,3.821070e+06,72.107888,223.686142,3.940493e+06,1.031254,1.021535,175.0,-323.0,0.009934,3.940492e+06,73.413136,0.009952
LINK-USD,81430.0,3.879533,0.082676,3.821070e+06,72.107888,4.158622,3.940493e+06,1.031254,1.071939,79051.0,-2379.0,0.083427,3.940492e+06,73.413136,0.083425
MIOTA-USD,252701.0,0.211438,0.013983,3.821070e+06,72.107888,0.219896,3.940493e+06,1.031254,1.040002,42765.0,-209936.0,0.002386,3.940492e+06,73.413136,0.002386
NANO-USD,479113.0,0.727661,0.091239,3.821070e+06,72.107888,0.752157,3.940493e+06,1.031254,1.033665,532298.0,53185.0,0.101605,3.940492e+06,73.413136,0.101603
OMG-USD,282352.0,0.846505,0.062551,3.821070e+06,72.107888,0.857160,3.940493e+06,1.031254,1.012587,219537.0,-62815.0,0.047755,3.940492e+06,73.413136,0.047754
XEM-USD,8769166.0,0.048902,0.112228,3.821070e+06,72.107888,0.049499,3.940493e+06,1.031254,1.012203,12237816.0,3468650.0,0.153726,3.940492e+06,73.413136,0.153723


# Re-escribimos columna Cantidad Nominal, aplicando el cambio de la columna ajuste y buscamos nueva data del otro mes

In [37]:
abril = dt.date(2020,4,1)
comienzo = abril - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, abril)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
abril = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,12839697.0,0.048052,0.156574,3.940492e+06,73.413136,0.030615,2.713808e+06,0.688698,0.637118,13879412.0,1039715.0,0.156577,2.713802e+06,80.288426,0.156572
DCR-USD,17003.0,17.618406,0.076022,3.940492e+06,73.413136,11.445779,2.713808e+06,0.688698,0.649649,18024.0,1021.0,0.076018,2.713802e+06,80.288426,0.076020
DGB-USD,106941092.0,0.005769,0.156574,3.940492e+06,73.413136,0.004469,2.713808e+06,0.688698,0.774660,95075732.0,-11865360.0,0.156577,2.713802e+06,80.288426,0.156572
DOGE-USD,271189565.0,0.002275,0.156574,3.940492e+06,73.413136,0.001812,2.713808e+06,0.688698,0.796276,234555589.0,-36633976.0,0.156577,2.713802e+06,80.288426,0.156572
ETH-USD,175.0,223.686142,0.009934,3.940492e+06,73.413136,132.560318,2.713808e+06,0.688698,0.592617,203.0,28.0,0.009916,2.713802e+06,80.288426,0.009941
LINK-USD,79051.0,4.158622,0.083427,3.940492e+06,73.413136,2.224416,2.713808e+06,0.688698,0.534893,101784.0,22733.0,0.083429,2.713802e+06,80.288426,0.083427
MIOTA-USD,42765.0,0.219896,0.002386,3.940492e+06,73.413136,0.154967,2.713808e+06,0.688698,0.704730,41793.0,-972.0,0.002387,2.713802e+06,80.288426,0.002386
NANO-USD,532298.0,0.752157,0.101605,3.940492e+06,73.413136,0.468589,2.713808e+06,0.688698,0.622994,588452.0,56154.0,0.101607,2.713802e+06,80.288426,0.101604
OMG-USD,219537.0,0.857160,0.047755,3.940492e+06,73.413136,0.510764,2.713808e+06,0.688698,0.595879,253740.0,34203.0,0.047756,2.713802e+06,80.288426,0.047755
XEM-USD,12237816.0,0.049499,0.153726,3.940492e+06,73.413136,0.036196,2.713808e+06,0.688698,0.731252,11525995.0,-711821.0,0.153731,2.713802e+06,80.288426,0.153726


In [38]:
mayo = dt.date(2020,5,1)
comienzo = mayo - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, mayo)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
mayo = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,13879412.0,0.030615,0.156577,2.713802e+06,80.288426,0.048575,4.664185e+06,1.71869,1.586630,16492145.0,2612733.0,0.171754,4.664242e+06,22.779893,0.171753
DCR-USD,18024.0,11.445779,0.076018,2.713802e+06,80.288426,13.935275,4.664185e+06,1.71869,1.217503,22260.0,4236.0,0.066506,4.664242e+06,22.779893,0.066507
DGB-USD,95075732.0,0.004469,0.156577,2.713802e+06,80.288426,0.015109,4.664185e+06,1.71869,3.380559,53022820.0,-42052912.0,0.171754,4.664242e+06,22.779893,0.171753
DOGE-USD,234555589.0,0.001812,0.156577,2.713802e+06,80.288426,0.002481,4.664185e+06,1.71869,1.369368,322929193.0,88373604.0,0.171754,4.664242e+06,22.779893,0.171753
ETH-USD,203.0,132.560318,0.009916,2.713802e+06,80.288426,211.422760,4.664185e+06,1.71869,1.594917,25.0,-178.0,0.001133,4.664242e+06,22.779893,0.001136
LINK-USD,101784.0,2.224416,0.083429,2.713802e+06,80.288426,3.759043,4.664185e+06,1.71869,1.689901,99381.0,-2403.0,0.080094,4.664242e+06,22.779893,0.080094
MIOTA-USD,41793.0,0.154967,0.002387,2.713802e+06,80.288426,0.184024,4.664185e+06,1.71869,1.187505,1661.0,-40132.0,0.000066,4.664242e+06,22.779893,0.000066
NANO-USD,588452.0,0.468589,0.101607,2.713802e+06,80.288426,0.630587,4.664185e+06,1.71869,1.345713,878720.0,290268.0,0.118799,4.664242e+06,22.779893,0.118799
OMG-USD,253740.0,0.510764,0.047756,2.713802e+06,80.288426,0.757247,4.664185e+06,1.71869,1.482578,161647.0,-92093.0,0.026244,4.664242e+06,22.779893,0.026244
XEM-USD,11525995.0,0.036196,0.153731,2.713802e+06,80.288426,0.041838,4.664185e+06,1.71869,1.155876,19147657.0,7621662.0,0.171754,4.664242e+06,22.779893,0.171753


In [39]:
junio = dt.date(2020,6,1)
comienzo = junio - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, junio)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
junio = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,16492145.0,0.048575,0.171754,4.664242e+06,22.779893,0.079441,5.760335e+06,1.234999,1.635440,12454115.0,-4038030.0,0.171758,5.760250e+06,107.187517,0.171755
DCR-USD,22260.0,13.935275,0.066506,4.664242e+06,22.779893,14.699932,5.760335e+06,1.234999,1.054872,26058.0,3798.0,0.066499,5.760250e+06,107.187517,0.066498
DGB-USD,53022820.0,0.015109,0.171754,4.664242e+06,22.779893,0.017178,5.760335e+06,1.234999,1.136976,57594621.0,4571801.0,0.171758,5.760250e+06,107.187517,0.171755
DOGE-USD,322929193.0,0.002481,0.171754,4.664242e+06,22.779893,0.002584,5.760335e+06,1.234999,1.041800,382818664.0,59889471.0,0.171758,5.760250e+06,107.187517,0.171755
ETH-USD,25.0,211.422760,0.001133,4.664242e+06,22.779893,237.768875,5.760335e+06,1.234999,1.124613,27.0,2.0,0.001114,5.760250e+06,107.187517,0.001131
LINK-USD,99381.0,3.759043,0.080094,4.664242e+06,22.779893,4.384033,5.760335e+06,1.234999,1.166263,105240.0,5859.0,0.080096,5.760250e+06,107.187517,0.080096
MIOTA-USD,1661.0,0.184024,0.000066,4.664242e+06,22.779893,0.224239,5.760335e+06,1.234999,1.218532,1683.0,22.0,0.000066,5.760250e+06,107.187517,0.000066
NANO-USD,878720.0,0.630587,0.118799,4.664242e+06,22.779893,0.878449,5.760335e+06,1.234999,1.393066,779036.0,-99684.0,0.118804,5.760250e+06,107.187517,0.118802
OMG-USD,161647.0,0.757247,0.026244,4.664242e+06,22.779893,1.503152,5.760335e+06,1.234999,1.985022,100573.0,-61074.0,0.026245,5.760250e+06,107.187517,0.026244
XEM-USD,19147657.0,0.041838,0.171754,4.664242e+06,22.779893,0.042502,5.760335e+06,1.234999,1.015860,23278341.0,4130684.0,0.171758,5.760250e+06,107.187517,0.171755


In [40]:
julio = dt.date(2020,7,1)
comienzo = julio - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, julio)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
julio = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,12454115.0,0.079441,0.171758,5.760250e+06,107.187517,0.083057,5.920043e+06,1.027741,1.045520,12956308.0,502193.0,1.817748e-01,5.920052e+06,97.604325,1.817718e-01
DCR-USD,26058.0,14.699932,0.066499,5.760250e+06,107.187517,14.063151,5.920043e+06,1.027741,0.956681,19595.0,-6463.0,4.654814e-02,5.920052e+06,97.604325,4.654955e-02
DGB-USD,57594621.0,0.017178,0.171758,5.760250e+06,107.187517,0.020443,5.920043e+06,1.027741,1.190084,52638709.0,-4955912.0,1.817748e-01,5.920052e+06,97.604325,1.817718e-01
DOGE-USD,382818664.0,0.002584,0.171758,5.760250e+06,107.187517,0.002314,5.920043e+06,1.027741,0.895347,465052970.0,82234306.0,1.817748e-01,5.920052e+06,97.604325,1.817718e-01
ETH-USD,27.0,237.768875,0.001114,5.760250e+06,107.187517,226.356750,5.920043e+06,1.027741,0.952003,0.0,-27.0,0.000000e+00,5.920052e+06,97.604325,1.351505e-05
LINK-USD,105240.0,4.384033,0.080096,5.760250e+06,107.187517,4.547113,5.920043e+06,1.027741,1.037199,87900.0,-17340.0,6.751482e-02,5.920052e+06,97.604325,6.751423e-02
MIOTA-USD,1683.0,0.224239,0.000066,5.760250e+06,107.187517,0.225105,5.920043e+06,1.027741,1.003862,1.0,-1682.0,3.802423e-08,5.920052e+06,97.604325,4.519847e-08
NANO-USD,779036.0,0.878449,0.118804,5.760250e+06,107.187517,0.924946,5.920043e+06,1.027741,1.052930,950696.0,171660.0,1.485362e-01,5.920052e+06,97.604325,1.485338e-01
OMG-USD,100573.0,1.503152,0.026245,5.760250e+06,107.187517,1.463972,5.920043e+06,1.027741,0.973934,29311.0,-71262.0,7.248326e-03,5.920052e+06,97.604325,7.248439e-03
XEM-USD,23278341.0,0.042502,0.171758,5.760250e+06,107.187517,0.042700,5.920043e+06,1.027741,1.004658,25201986.0,1923645.0,1.817748e-01,5.920052e+06,97.604325,1.817718e-01


In [41]:
agosto = dt.date(2020,8,1)
comienzo = agosto - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, agosto)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
agosto = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,12956308.0,0.083057,1.817748e-01,5.920052e+06,97.604325,0.141646,7.911921e+06,1.336461,1.705396,10153417.0,-2802891.0,1.817751e-01,7.911905e+06,113.495554,1.817725e-01
DCR-USD,19595.0,14.063151,4.654814e-02,5.920052e+06,97.604325,15.395735,7.911921e+06,1.336461,1.094757,23916.0,4321.0,4.653802e-02,7.911905e+06,113.495554,4.653818e-02
DGB-USD,52638709.0,0.020443,1.817748e-01,5.920052e+06,97.604325,0.024354,7.911921e+06,1.336461,1.191263,59054609.0,6415900.0,1.817751e-01,7.911905e+06,113.495554,1.817725e-01
DOGE-USD,465052970.0,0.002314,1.817748e-01,5.920052e+06,97.604325,0.003215,7.911921e+06,1.336461,1.389470,447310862.0,-17742108.0,1.817751e-01,7.911905e+06,113.495554,1.817725e-01
ETH-USD,0.0,226.356750,0.000000e+00,5.920052e+06,97.604325,346.265961,7.911921e+06,1.336461,1.529736,0.0,0.0,0.000000e+00,7.911905e+06,113.495554,1.307176e-05
LINK-USD,87900.0,4.547113,6.751482e-02,5.920052e+06,97.604325,7.781163,7.911921e+06,1.336461,1.711231,68651.0,-19249.0,6.751657e-02,7.911905e+06,113.495554,6.751588e-02
MIOTA-USD,1.0,0.225105,3.802423e-08,5.920052e+06,97.604325,0.300729,7.911921e+06,1.336461,1.335947,1.0,0.0,3.800966e-08,7.911905e+06,113.495554,4.517404e-08
NANO-USD,950696.0,0.924946,1.485362e-01,5.920052e+06,97.604325,0.942009,7.911921e+06,1.336461,1.018448,1247604.0,296908.0,1.485425e-01,7.911905e+06,113.495554,1.485405e-01
OMG-USD,29311.0,1.463972,7.248326e-03,5.920052e+06,97.604325,1.657051,7.911921e+06,1.336461,1.131888,34611.0,5300.0,7.248849e-03,7.911905e+06,113.495554,7.248788e-03
XEM-USD,25201986.0,0.042700,1.817748e-01,5.920052e+06,97.604325,0.053279,7.911921e+06,1.336461,1.247764,26993454.0,1791468.0,1.817751e-01,7.911905e+06,113.495554,1.817725e-01


In [42]:
septiembre = dt.date(2020,9,1)
comienzo = septiembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, septiembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
septiembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,10153417.0,0.141646,1.817751e-01,7.911905e+06,113.495554,0.123600,1.125080e+07,1.422009,0.872602,17010479.0,6857062.0,1.868739e-01,1.125089e+07,18.577925,1.868736e-01
DCR-USD,23916.0,15.395735,4.653802e-02,7.911905e+06,113.495554,17.239244,1.125080e+07,1.422009,1.119742,13676.0,-10240.0,2.095513e-02,1.125089e+07,18.577925,2.095554e-02
DGB-USD,59054609.0,0.024354,1.817751e-01,7.911905e+06,113.495554,0.026959,1.125080e+07,1.422009,1.106987,77988694.0,18934085.0,1.868739e-01,1.125089e+07,18.577925,1.868736e-01
DOGE-USD,447310862.0,0.003215,1.817751e-01,7.911905e+06,113.495554,0.003225,1.125080e+07,1.422009,1.003046,651941558.0,204630696.0,1.868739e-01,1.125089e+07,18.577925,1.868736e-01
ETH-USD,0.0,346.265961,0.000000e+00,7.911905e+06,113.495554,436.700165,1.125080e+07,1.422009,1.261170,0.0,0.0,0.000000e+00,1.125089e+07,18.577925,0.000000e+00
LINK-USD,68651.0,7.781163,6.751657e-02,7.911905e+06,113.495554,15.839754,1.125080e+07,1.422009,2.035654,31313.0,-37338.0,4.408452e-02,1.125089e+07,18.577925,4.408515e-02
MIOTA-USD,1.0,0.300729,3.800966e-08,7.911905e+06,113.495554,0.362098,1.125080e+07,1.422009,1.204068,0.0,-1.0,0.000000e+00,1.125089e+07,18.577925,1.398347e-14
NANO-USD,1247604.0,0.942009,1.485425e-01,7.911905e+06,113.495554,1.132777,1.125080e+07,1.422009,1.202511,1856057.0,608453.0,1.868739e-01,1.125089e+07,18.577925,1.868736e-01
OMG-USD,34611.0,1.657051,7.248849e-03,7.911905e+06,113.495554,5.082007,1.125080e+07,1.422009,3.066898,1124.0,-33487.0,5.077087e-04,1.125089e+07,18.577925,5.081238e-04
XEM-USD,26993454.0,0.053279,1.817751e-01,7.911905e+06,113.495554,0.142570,1.125080e+07,1.422009,2.675905,14747146.0,-12246308.0,1.868739e-01,1.125089e+07,18.577925,1.868736e-01


In [43]:
octubre = dt.date(2020,10,1)
comienzo = octubre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, octubre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
octubre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,17010479.0,0.123600,1.868739e-01,1.125089e+07,18.577925,0.103848,9.370219e+06,0.832842,0.840189,16861863.0,-148616.0,1.868754e-01,9.370218e+06,18.80241,1.868750e-01
DCR-USD,13676.0,17.239244,2.095513e-02,1.125089e+07,18.577925,11.813062,9.370219e+06,0.832842,0.685242,16614.0,2938.0,2.094532e-02,9.370218e+06,18.80241,2.094636e-02
DGB-USD,77988694.0,0.026959,1.868739e-01,1.125089e+07,18.577925,0.027201,9.370219e+06,0.832842,1.008988,64374187.0,-13614507.0,1.868754e-01,9.370218e+06,18.80241,1.868750e-01
DOGE-USD,651941558.0,0.003225,1.868739e-01,1.125089e+07,18.577925,0.002664,9.370219e+06,0.832842,0.826188,657197358.0,5255800.0,1.868754e-01,9.370218e+06,18.80241,1.868750e-01
ETH-USD,0.0,436.700165,0.000000e+00,1.125089e+07,18.577925,363.187866,9.370219e+06,0.832842,0.831664,0.0,0.0,0.000000e+00,9.370218e+06,18.80241,0.000000e+00
LINK-USD,31313.0,15.839754,4.408452e-02,1.125089e+07,18.577925,10.018506,9.370219e+06,0.832842,0.632491,41234.0,9921.0,4.408681e-02,9.370218e+06,18.80241,4.408730e-02
MIOTA-USD,0.0,0.362098,0.000000e+00,1.125089e+07,18.577925,0.286845,9.370219e+06,0.832842,0.792175,0.0,0.0,0.000000e+00,9.370218e+06,18.80241,1.397271e-14
NANO-USD,1856057.0,1.132777,1.868739e-01,1.125089e+07,18.577925,0.829130,9.370219e+06,0.832842,0.731944,2111929.0,255872.0,1.868753e-01,9.370218e+06,18.80241,1.868750e-01
OMG-USD,1124.0,5.082007,5.077087e-04,1.125089e+07,18.577925,4.167864,9.370219e+06,0.832842,0.820122,1142.0,18.0,5.079605e-04,9.370218e+06,18.80241,5.081956e-04
XEM-USD,14747146.0,0.142570,1.868739e-01,1.125089e+07,18.577925,0.117018,9.370219e+06,0.832842,0.820774,14964092.0,216946.0,1.868754e-01,9.370218e+06,18.80241,1.868750e-01


In [44]:
noviembre = dt.date(2020,11,1)
comienzo = noviembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, noviembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
noviembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,16861863.0,0.103848,1.868754e-01,9.370218e+06,18.80241,0.093694,8.232940e+06,0.878628,0.902225,17188491.0,326628.0,1.956111e-01,8.232946e+06,12.808587,1.956109e-01
DCR-USD,16614.0,11.813062,2.094532e-02,9.370218e+06,18.80241,12.306911,8.232940e+06,0.878628,1.041805,2705.0,-13909.0,4.043533e-03,8.232946e+06,12.808587,4.044254e-03
DGB-USD,64374187.0,0.027201,1.868754e-01,9.370218e+06,18.80241,0.019962,8.232940e+06,0.878628,0.733878,80674264.0,16300077.0,1.956112e-01,8.232946e+06,12.808587,1.956109e-01
DOGE-USD,657197358.0,0.002664,1.868754e-01,9.370218e+06,18.80241,0.002565,8.232940e+06,0.878628,0.962641,627882947.0,-29314411.0,1.956112e-01,8.232946e+06,12.808587,1.956109e-01
ETH-USD,0.0,363.187866,0.000000e+00,9.370218e+06,18.80241,388.037720,8.232940e+06,0.878628,1.068421,0.0,0.0,0.000000e+00,8.232946e+06,12.808587,0.000000e+00
LINK-USD,41234.0,10.018506,4.408681e-02,9.370218e+06,18.80241,11.220458,8.232940e+06,0.878628,1.119973,13133.0,-28101.0,1.789861e-02,8.232946e+06,12.808587,1.789905e-02
MIOTA-USD,0.0,0.286845,0.000000e+00,9.370218e+06,18.80241,0.251149,8.232940e+06,0.878628,0.875557,0.0,0.0,0.000000e+00,8.232946e+06,12.808587,0.000000e+00
NANO-USD,2111929.0,0.829130,1.868753e-01,9.370218e+06,18.80241,0.740834,8.232940e+06,0.878628,0.893508,2173842.0,61913.0,1.956111e-01,8.232946e+06,12.808587,1.956109e-01
OMG-USD,1142.0,4.167864,5.079605e-04,9.370218e+06,18.80241,2.918442,8.232940e+06,0.878628,0.700225,6.0,-1136.0,2.126900e-06,8.232946e+06,12.808587,2.374034e-06
XEM-USD,14964092.0,0.117018,1.868754e-01,9.370218e+06,18.80241,0.096666,8.232940e+06,0.878628,0.826084,16659926.0,1695834.0,1.956112e-01,8.232946e+06,12.808587,1.956109e-01


In [45]:
diciembre = dt.date(2020,12,1)
comienzo = diciembre - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, diciembre)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
diciembre = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,17188491.0,0.093694,1.956111e-01,8.232946e+06,12.808587,0.170053,1.310144e+07,1.591343,1.814991,15070512.0,-2117979.0,1.956118e-01,1.310142e+07,33.626764,1.956113e-01
DCR-USD,2705.0,12.306911,4.043533e-03,8.232946e+06,12.808587,24.797695,1.310144e+07,1.591343,2.014941,2134.0,-571.0,4.039125e-03,1.310142e+07,33.626764,4.040419e-03
DGB-USD,80674264.0,0.019962,1.956112e-01,8.232946e+06,12.808587,0.023257,1.310144e+07,1.591343,1.165015,110196526.0,29522262.0,1.956118e-01,1.310142e+07,33.626764,1.956113e-01
DOGE-USD,627882947.0,0.002565,1.956112e-01,8.232946e+06,12.808587,0.003541,1.310144e+07,1.591343,1.380724,723663067.0,95780120.0,1.956118e-01,1.310142e+07,33.626764,1.956113e-01
ETH-USD,0.0,388.037720,0.000000e+00,8.232946e+06,12.808587,610.587646,1.310144e+07,1.591343,1.573527,0.0,0.0,0.000000e+00,1.310142e+07,33.626764,0.000000e+00
LINK-USD,13133.0,11.220458,1.789861e-02,8.232946e+06,12.808587,14.295212,1.310144e+07,1.591343,1.274031,16405.0,3272.0,1.789981e-02,1.310142e+07,33.626764,1.790070e-02
MIOTA-USD,0.0,0.251149,0.000000e+00,8.232946e+06,12.808587,0.350853,1.310144e+07,1.591343,1.396991,0.0,0.0,0.000000e+00,1.310142e+07,33.626764,0.000000e+00
NANO-USD,2173842.0,0.740834,1.956111e-01,8.232946e+06,12.808587,1.198408,1.310144e+07,1.591343,1.617647,2138498.0,-35344.0,1.956118e-01,1.310142e+07,33.626764,1.956113e-01
OMG-USD,6.0,2.918442,2.126900e-06,8.232946e+06,12.808587,3.924593,1.310144e+07,1.591343,1.344756,7.0,1.0,2.096883e-06,1.310142e+07,33.626764,2.376361e-06
XEM-USD,16659926.0,0.096666,1.956112e-01,8.232946e+06,12.808587,0.193199,1.310144e+07,1.591343,1.998620,13265008.0,-3394918.0,1.956118e-01,1.310142e+07,33.626764,1.956113e-01


In [46]:
enero = dt.date(2021,1,1)
comienzo = enero - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, enero)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
enero = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,15070512.0,0.170053,1.956118e-01,1.310142e+07,33.626764,0.183858,1.452193e+07,1.108424,1.081177,15750827.0,680315.0,0.199417,1.452194e+07,30.215544,1.994161e-01
DCR-USD,2134.0,24.797695,4.039125e-03,1.310142e+07,33.626764,40.681427,1.452193e+07,1.108424,1.640533,50.0,-2084.0,0.000140,1.452194e+07,30.215544,1.417686e-04
DGB-USD,110196526.0,0.023257,1.956118e-01,1.310142e+07,33.626764,0.025106,1.452193e+07,1.108424,1.079537,115346015.0,5149489.0,0.199417,1.452194e+07,30.215544,1.994161e-01
DOGE-USD,723663067.0,0.003541,1.956118e-01,1.310142e+07,33.626764,0.004750,1.452193e+07,1.108424,1.341264,609669083.0,-113993984.0,0.199417,1.452194e+07,30.215544,1.994161e-01
ETH-USD,0.0,610.587646,0.000000e+00,1.310142e+07,33.626764,745.077332,1.452193e+07,1.108424,1.220263,0.0,0.0,0.000000,1.452194e+07,30.215544,0.000000e+00
LINK-USD,16405.0,14.295212,1.789981e-02,1.310142e+07,33.626764,11.600221,1.452193e+07,1.108424,0.811476,3477.0,-12928.0,0.002777,1.452194e+07,30.215544,2.777772e-03
MIOTA-USD,0.0,0.350853,0.000000e+00,1.310142e+07,33.626764,0.300068,1.452193e+07,1.108424,0.855252,0.0,0.0,0.000000,1.452194e+07,30.215544,0.000000e+00
NANO-USD,2138498.0,1.198408,1.956118e-01,1.310142e+07,33.626764,1.038870,1.452193e+07,1.108424,0.866875,2787560.0,649062.0,0.199416,1.452194e+07,30.215544,1.994161e-01
OMG-USD,7.0,3.924593,2.096883e-06,1.310142e+07,33.626764,2.494372,1.452193e+07,1.108424,0.635575,0.0,-7.0,0.000000,1.452194e+07,30.215544,3.922404e-11
XEM-USD,13265008.0,0.193199,1.956118e-01,1.310142e+07,33.626764,0.229803,1.452193e+07,1.108424,1.189457,12601746.0,-663262.0,0.199416,1.452194e+07,30.215544,1.994161e-01


In [47]:
febrero = dt.date(2021,2,1)
comienzo = febrero - dt.timedelta(90)
update = AdjustRisk(portfolio, comienzo, febrero)

portfolio['nominal'] = portfolio['nominalNew'].values
portfolio['pricePaid'] = portfolio['priceToday'].values
portfolio['weights'] = (portfolio['nominal'] * portfolio['pricePaid']) / sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['notionalStart'] = sum(portfolio['nominal'] * portfolio['pricePaid'])
portfolio['oldLiquidity'] = portfolio['liquidityToReinvest'].values
stocks = list(portfolio.index)
portfolio['priceToday'] = update['lastPrice'].values
portfolio['notionalToday'] = sum(portfolio['priceToday'] * portfolio['nominal'])
portfolio['PnLpercent'] = portfolio['notionalToday'] / portfolio['notionalStart']
portfolio['PnLpercentEach'] = portfolio['priceToday'] / portfolio['pricePaid']
portfolio['updatedRisk'] = (update['MinCVaR'].values)
# En nuevo nominal sumamos el resultado obtenido mas el remanente liquido para reinvertir, siendo nuestro total disponible
portfolio['nominalNew'] = (portfolio['updatedRisk'] * (portfolio['notionalToday'] + portfolio['oldLiquidity']) // portfolio['priceToday']) # nuevo nominal
portfolio['adjust'] = portfolio['nominalNew'] - portfolio['nominal'] # ajuste nominal
portfolio['percentReb'] = (portfolio['nominalNew'] * portfolio['priceToday']) / sum(portfolio['nominalNew'] * portfolio['priceToday'])
# Columnas vinculantes para conectar mes anterior con el proximo ya armado
portfolio['notionalRebalance'] = sum(portfolio['nominalNew'] * portfolio['priceToday'])
portfolio['liquidityToReinvest'] =  (portfolio['notionalToday'] + portfolio['oldLiquidity']) - portfolio['notionalRebalance']
febrero = copy.copy(portfolio)
portfolio

[*********************100%***********************]  12 of 12 completed


,nominal,pricePaid,weights,notionalStart,oldLiquidity,priceToday,notionalToday,PnLpercent,PnLpercentEach,nominalNew,adjust,percentReb,notionalRebalance,liquidityToReinvest,updatedRisk
ADA-USD,15750827.0,0.183858,0.199417,1.452194e+07,30.215544,0.344674,4.431942e+07,3.051895,1.874677,25641642.0,9890815.0,0.199416,4.431939e+07,62.763845,1.994161e-01
DCR-USD,50.0,40.681427,0.000140,1.452194e+07,30.215544,66.985100,4.431942e+07,3.051895,1.646577,93.0,43.0,0.000141,4.431939e+07,62.763845,1.414587e-04
DGB-USD,115346015.0,0.025106,0.199417,1.452194e+07,30.215544,0.028685,4.431942e+07,3.051895,1.142522,308110745.0,192764730.0,0.199416,4.431939e+07,62.763845,1.994161e-01
DOGE-USD,609669083.0,0.004750,0.199417,1.452194e+07,30.215544,0.037697,4.431942e+07,3.051895,7.936159,234451350.0,-375217733.0,0.199416,4.431939e+07,62.763845,1.994161e-01
ETH-USD,0.0,745.077332,0.000000,1.452194e+07,30.215544,1316.760498,4.431942e+07,3.051895,1.767280,0.0,0.0,0.000000,4.431939e+07,62.763845,0.000000e+00
LINK-USD,3477.0,11.600221,0.002777,1.452194e+07,30.215544,22.407177,4.431942e+07,3.051895,1.931616,5494.0,2017.0,0.002778,4.431939e+07,62.763845,2.778126e-03
MIOTA-USD,0.0,0.300068,0.000000,1.452194e+07,30.215544,0.403463,4.431942e+07,3.051895,1.344572,0.0,0.0,0.000000,4.431939e+07,62.763845,0.000000e+00
NANO-USD,2787560.0,1.038870,0.199416,1.452194e+07,30.215544,3.420964,4.431942e+07,3.051895,3.292965,2583485.0,-204075.0,0.199416,4.431939e+07,62.763845,1.994161e-01
OMG-USD,0.0,2.494372,0.000000,1.452194e+07,30.215544,3.505735,4.431942e+07,3.051895,1.405458,0.0,0.0,0.000000,4.431939e+07,62.763845,3.812446e-11
XEM-USD,12601746.0,0.229803,0.199416,1.452194e+07,30.215544,0.236637,4.431942e+07,3.051895,1.029740,37348395.0,24746649.0,0.199416,4.431939e+07,62.763845,1.994161e-01
